In [ ]:
# Correlation Analysis

In [ ]:
%matplotlib inline
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import datetime

In [ ]:
def runcorr(d):
    # ticker1 and ticker2 are global widgets
    global combined_data
    combined_data = corr(ticker1.value, ticker2.value)
    return None

In [ ]:
def corr(ticker1, ticker2):
    """Shows total correlation of two financial instruments over entire history"""
    # Download data
    price_data_1 = yf.download(ticker1)
    price_data_2 = yf.download(ticker2)
    price_data_2 = price_data_2.rename(columns = {"Open": "Open", "High": "High", "Low": "Low", "Close": "Close", "Adj Close":"Adj Close 2", "Volume": "Volume"})
    combined_data = pd.merge(price_data_1["Adj Close"], price_data_2["Adj Close 2"], left_index = True, right_index = True)
    # Take log returns of daily price change
    adj_data_1 = np.log(combined_data.iloc[1:]["Adj Close"].values/combined_data.iloc[:-1]["Adj Close"].values)
    adj_data_2 = np.log(combined_data.iloc[1:]["Adj Close 2"].values/combined_data.iloc[:-1]["Adj Close 2"].values)
    # Combine data 
    combined_data.loc[combined_data.index[1]:, "Adj Close"] = adj_data_1
    combined_data.loc[combined_data.index[1]:, "Adj Close 2"] = adj_data_2
    combined_data = combined_data.dropna().loc[combined_data.index[1]:,:]
    
    # Print data
    out.clear_output() # out is a global widget
    with out:
        print("Correlation:",combined_data.corr().iloc[0,1])
    return combined_data

In [ ]:
def runintc(days):
    print(intcorr(combined_data, days))

In [ ]:
def intcorr(combined_data, days):
    """
    Shows total correlation of two financial instruments over a defined
    number of trading days
    """
#     # Download data
#     price_data_2 = price_data_2.rename(columns = {"Open": "Open", "High": "High", "Low": "Low", "Close": "Close", "Adj Close":"Adj Close 2", "Volume": "Volume"})
#     combined_data = pd.merge(price_data_1["Adj Close"], price_data_2["Adj Close 2"], left_index = True, right_index = True)
#     # Take log returns of daily price change
#     adj_data_1 = np.log(combined_data.iloc[1:]["Adj Close"].values/combined_data.iloc[:-1]["Adj Close"].values)
#     adj_data_2 = np.log(combined_data.iloc[1:]["Adj Close 2"].values/combined_data.iloc[:-1]["Adj Close 2"].values)
#     # Combine data 
#     combined_data.loc[combined_data.index[1]:, "Adj Close"] = adj_data_1
#     combined_data.loc[combined_data.index[1]:, "Adj Close 2"] = adj_data_2
#     combined_data = combined_data.dropna().loc[combined_data.index[1]:,:]
    
    # Return correlation
    if days == 0:
        return f"All time correlation: {combined_data.iloc[-days:].corr().iloc[0,1]}"
    else:
        return f"Correlation in the last {days} trading days: {combined_data.iloc[-days:].corr().iloc[0,1]}"

In [ ]:
def runplotcorr(window):
    plotcorr(combined_data, window)

In [ ]:
def plotcorr(combined_data, window):
    """
    Plots trailing correlation over time
    """
#     times = combined_data.index
#     date_window = datetime.timedelta(days = window)
#     corrs = []
#     # Calculate correlation within all windows
#     for i in combined_data[times[0] + date_window:].index:
#         corrs.append([i, combined_data[i - date_window:i].corr().iloc[0][1]])
    corrs = combined_data["Adj Close"].rolling(window).corr(other = combined_data["Adj Close 2"],pairwise = False)
    
    # Plot entire history of correlation coefficient
    
    fig = plt.figure(figsize = (14, 8))
    ax = plt.axes()
    
    ax.set_ylabel("Correlation Coefficient")
    ax.yaxis.label.set_size(18)

    ax.plot(corrs);
    
    return None
    
    

In [ ]:
# Get Historic Correlation
ticker1 = widgets.Text(placeholder = "Enter ticker symbol", description = "Ticker 1")
ticker2 = widgets.Text(placeholder = "Enter ticker symbol", description = "Ticker 2")
row = widgets.HBox([ticker1, ticker2])
go = widgets.Button(description = "Get historic correlation",
                   disabled = False, button_style = '',
                   tooltip = 'Click to go',
                   layout = {'margin': '0px 0px 0px 70px', 'display': 'flex', 'border': '1px solid black'}
                   )
out = widgets.Output(layout = {'border': '1px solid black'})
display(row)
display(go)
display(out)
go.on_click(runcorr)

In [ ]:
# Get Correlation in the last x trading days

days = widgets.IntSlider(description = "Days", 
                         min = 0, max = len(combined_data.dropna()),
                        layout = {'width': '750px'})
out = widgets.interactive_output(runintc, {'days': days})

widgets.VBox([days, out])

In [ ]:
# See correlation evolving over time
window = widgets.IntSlider(description = "", 
                         min = 60, max = min(506,len(combined_data.dropna())),
                        layout = {'width': '500px'})

out = widgets.interactive_output(runplotcorr, {'window': window})

widgets.VBox([widgets.HBox([widgets.Label("Window in trading days"), window]), out])